In [13]:
import re

import pandas as pd
from docx import Document
from word2number import w2n

In [14]:
SCRIPT = "../Clean Hands.docx"
PREAMBLE_LEN = 4

In [15]:
document = Document(SCRIPT)

In [16]:
lines = []
for p in document.paragraphs:
    for run in p.runs:
        text = run.text.strip()
        if not text:
            continue
        if run.italic:
            if lines and text.endswith(":"):
                lines[-1] += ":"
            continue
        if text == ":":
            lines[-1] += ":"
            continue
        if text.startswith(": "):
            lines[-1] += ":"
            text = text[2:]
        print(f"text: {text} hello {len(text)}")
        lines.extend(text.replace(": ", ":\n").splitlines())

text: Jovan Hrisitić hello 14
text: CLEAN HANDS hello 11
text: Adaptation: Tom Celeketic hello 25
text: ACT ONE hello 7
text: SPHINX: 
Get lost. hello 18
text: BOY: 
Why? hello 10
text: SPHINX: 
You're bothering me; go away. hello 38
text: BOY: 
You killed that old man with the sheep. hello 45
text: SPHINX: 
That doesn't mean you can bore me. Go play somewhere else. hello 67
text: BOY: 
You go somewhere else. We never called you to come. You killed that old man; isn't that enough for you? hello 109
text: SPHINX:
 What do I get from killing that old man? hello 49
text: BOY: 
Then why did you stop him? You could have let him bring his sheep to town. It would have been better for all of us. hello 121
text: SPHINX: 
I had to stop him, as I stopped everyone who passed this road. I'm like a doctor; he can't choose his patients either. hello 127
text: BOY: 
Do you think it's nice to kill people in the middle of the road? That's not nice. Have you never thought about it? hello 120
text: SPHINX

In [17]:
lines = [
    line.strip()
    for line in lines
]

In [18]:
act = -1
character = "NO ONE"
lines_parsed = []
current_line = []

for line in lines[PREAMBLE_LEN:]:
    if "THE END" in line:
        if current_line:
            lines_parsed.append((character, " ".join(current_line), act))   
        break  
    if line.startswith("(") and line.endswith(")"):
        continue
    if line.startswith("ACT"):
        act = w2n.word_to_num(line.split(" ")[1])
        continue
    if line.endswith(":") and line[:-1].isupper()]:
        if current_line:
            lines_parsed.append((character, " ".join(current_line), act))
            current_line = []
        character = line[:-1]
        continue
    if line.startswith("-"):
        if current_line:
            lines_parsed.append((character, " ".join(current_line), act))
            current_line = []
        line = line[1:].strip()
        character = "CHOIR"
    current_line.append(line)

lines_df = pd.DataFrame(lines_parsed, columns=["character", "line", "act"])

In [19]:
lines_df["line"] = lines_df["line"].apply(str.strip)
lines_df["line_no"] = lines_df.index + 1

In [20]:
lines_df.to_json("lines.json", orient="records", indent=2, index=True)